#Wazy + ColabFold for Finding Peptide Binders

This notebook is used to find a peptide binder for a given protein. Affinity is assessed based on predicted distance to target binding site and AlphaFold2 confidence of peptide's structure.

## Quickstart

Enter your the protein sequence for what you want to bind to and optionally specify the residues you want to bind in the form below. Then go to Menu Runtime -> Run All. You can re-execute the "Perform Optimization Cell" additional times to increase number of optimization rounds.

## Important Notes:##

* Amber Relax should be used for best accuracy
* Recycles should be 6 or greater
* Results will be printed, plotting, and can be downloaded in the `bo-results.csv` file.

## About Wazy

Wazy is a method for optimizing sequences for a numeric task, like quantitative activity or solubility. Wazy uses Bayesian Optimization to propose which new sequences should be tried. The method is designed for when you have few (1-100) starting sequences and want to know which additional sequences to try in order to find the best. **In this notebook, wazy is being used to optimize a sequence for binding to a protein according to the AlphaFold-Multimer model. AlphaFold-Multimer is the black box model. The notebook handles testing newly proposed sequences and updating surrogate model.** See the [wazy paper](https://www.biorxiv.org/content/10.1101/2022.08.05.502972v1) and the [code](https://github.com/ur-whitelab/wazy) for complete details on how the algorithm works. This notebook was forked from [ColabFold](https://github.com/sokrypton/ColabFold). The main change was inserting Wazy code and removing custom MSA options (for simplicity).


Credit:

* [AlphaFold2 Paper](https://www.nature.com/articles/s41586-021-03819-2), [Multimer Paper](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v2)
* This doc authored by [@andrewwhite01](https://twitter.com/andrewwhite01)
* Wazy authored by [@andrewwhite01](https://twitter.com/andrewwhite01) and [@ZiyueYang37](https://twitter.com/ZiyueYang37)
* [Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all. *Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1) 

In [ ]:
#@title Input protein sequence(s), then hit `Runtime` -> `Run all`
from google.colab import files
import os.path
import re
import hashlib
import random

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

protein_sequence = 'GMTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEASAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQH' #@param {type:"string"}
starting_binder = 'GGGGGGGGGGGGGGG' #@param {type:"string"}
variable_length = True #@param {type:"boolean"}
min_peptide_length = 6 #@param {type:"integer"}
#@markdown * Should the peptide binder length vary
binding_site = 'resid 25 to 40' #@param {type:"string"}
#@markdown * MDAnalysis selection string. Leave empty if you don't care where it binds. Example: *resid 5 to 25*

# remove whitespaces
protein_sequence = "".join(protein_sequence.split())

jobname = 'wazy' #@param {type:"string"}
# remove whitespaces
basejobname = "".join(jobname.split())
basejobname = re.sub(r'\W+', '', basejobname)

# number of models to use
use_amber = False #@param {type:"boolean"}
template_mode = "pdb70" #@param ["none", "pdb70", "custom"]
#@markdown - "none" = no template information is used, "pdb70" = detect templates in pdb70, "custom" - upload and search own templates (PDB or mmCIF format, see [notes below](#custom_templates))

if template_mode == "pdb70":
  use_templates = True
  custom_template_path = None
elif template_mode == "custom":
  custom_template_path = f"{jobname}_template"
  os.mkdir(custom_template_path)
  uploaded = files.upload()
  use_templates = True
  for fn in uploaded.keys():
    os.rename(fn, f"{jobname}_template/{fn}")
else:
  custom_template_path = None
  use_templates = False


In [ ]:
#@markdown ### MSA options (custom MSA upload, single sequence, pairing mode)
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence"]
pair_mode = "unpaired+paired" #@param ["unpaired+paired","paired","unpaired"] {type:"string"}
#@markdown - "unpaired+paired" = pair sequences from same species + unpaired MSA, "unpaired" = seperate MSA for each chain, "paired" - only use paired sequences.

In [ ]:
#@markdown ### Advanced settings
model_type = "AlphaFold2-multimer-v2"
num_models = 1 #@param {type:"slider", min:1, max:5, step:1}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
save_to_google_drive = False #@param {type:"boolean"}
seed = 0 #@param {type:"integer"}

#@markdown -  if the save_to_google_drive option was selected, the result zip will be uploaded to your Google Drive
dpi = 200 

#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form.


if save_to_google_drive:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("You are logged into Google Drive and are good to go!")

In [ ]:
#@title Install dependencies
%%bash -s $use_amber $use_templates

set -e

USE_AMBER=$1
USE_TEMPLATES=$2

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  # high risk high gain
  pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" wazy MDAnalysis -f https://storage.googleapis.com/jax-releases/jax_releases.html  
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

In [ ]:
#@title Set-up AlphaFold
#@markdown **WARNING: Running this a second time will remove all cached results!**

import sys, json, glob
import urllib.request
from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
if "1" in K80_chk:
  print("WARNING: found GPU Tesla K80: limited to total length < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import MDAnalysis as mda
import MDAnalysis.analysis.distances as mda_dist
import wazy
import jax
from functools import lru_cache

urllib.request.urlretrieve(
    "https://github.com/google/fonts/raw/main/ofl/courierprime/CourierPrime-Regular.ttf",
    "CourierPrime-Regular.ttf",
)
fe = mpl.font_manager.FontEntry(fname="CourierPrime-Regular.ttf", name="courierprime")
mpl.font_manager.fontManager.ttflist.append(fe)
color_cycle = ["#444444", "#1BBC9B", "#a895bb", "#F06060", "#F3B562", "#80cedb"]
mpl.rcParams.update(
    {
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler(color=color_cycle),
        "font.family": fe.name,
        "font.size": 14,
        "figure.figsize": (4.5, 4.5 / 1.3),
        "figure.dpi": 100,
        "ytick.left": True,
        "xtick.bottom": True,
        "image.cmap": "gist_yarg",
        "lines.markersize": 6,
    }
)


np.random.seed(seed)
key = jax.random.PRNGKey(seed)
boa = wazy.BOAlgorithm()
bo_results = []

# For some reason we need that to get pdbfixer to import
if use_amber and '/usr/local/lib/python3.7/site-packages/' not in sys.path:
    sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')

result_dir="."
if 'logging_setup' not in globals():
    setup_logging(Path(".").joinpath("log.txt"))
    logging_setup = True

@lru_cache(maxsize=1024)
def label_sequence(seq, model_type=model_type, af_model=1):
  query_sequence = protein_sequence + ':' + seq
  jobname = add_hash(basejobname, query_sequence)
  while os.path.isfile(f"{jobname}.csv"):
    jobname = add_hash(basejobname, ''.join(random.sample(query_sequence,len(query_sequence))))

  with open(f"{jobname}.csv", "w") as text_file:
      text_file.write(f"id,sequence\n{jobname},{query_sequence}")

  queries_path=f"{jobname}.csv"

  # decide which a3m to use
  if msa_mode.startswith("MMseqs2"):
    a3m_file = f"{jobname}.a3m"
  else:
    a3m_file = f"{jobname}.single_sequence.a3m"
    with open(a3m_file, "w") as text_file:
      text_file.write(">1\n%s" % query_sequence)

  queries, is_complex = get_queries(queries_path)
  model_type = set_model_type(is_complex, model_type)
  download_alphafold_params(model_type, Path("."))
  run(
      queries=queries,
      result_dir=result_dir,
      use_templates=use_templates,
      custom_template_path=custom_template_path,
      use_amber=use_amber,
      msa_mode=msa_mode,    
      model_type=model_type,
      num_models=1,
      num_recycles=num_recycles,
      model_order=[af_model],
      is_complex=is_complex,
      data_dir=Path("."),
      keep_existing_results=False,
      recompile_padding=1.0,
      rank_by="auto",
      pair_mode=pair_mode,
      stop_at_score=float(100)
  )
  rank_num = 1
  try:
    f = open(f"{jobname}_relaxed_rank_{rank_num}_model_1_scores.json")
  except FileNotFoundError as e:
    f = open(f"{jobname}_unrelaxed_rank_{rank_num}_model_1_scores.json")
  data = json.load(f)
  avg_plddt = np.mean(data["plddt"][len(protein_sequence):])

  pdb_filename = f"{jobname}_{'' if use_amber else 'un'}relaxed_rank_{rank_num}_model_*.pdb"
  pdb_file = glob.glob(pdb_filename)[0]  
  # load and find smallest chain
  u = mda.Universe(pdb_file)
  peptide = None
  for chain in u.segments:
      if peptide is None or len(chain.residues) < len(peptide):
          peptide = chain.residues
  protein = u.select_atoms(f'{binding_site} and not segid {peptide.segids[0]} and not name H*')
  peptide = peptide.atoms.select_atoms('not name H*')
  all_d = []
  for r in peptide.residues:
      distances = mda_dist.distance_array(r.atoms.positions, protein.positions)
      # get row, column of minimum distance
      i, j = np.unravel_index(distances.argmin(), distances.shape)
      all_d.append(distances[i,j])
  avg_d = np.mean(all_d)
  
  score = avg_plddt / 50 - avg_d / 10
  return {'seq': seq, 'score': score, 'rmsd': avg_d, 'plddt': avg_plddt, 'id': jobname}
# will be cached, so we aren't wasting time here
_ = label_sequence(starting_binder)

In [ ]:
#@title Perform optimization. Re-execute as many times as desired.
iterations = 50 #@param {type:"integer"}

seq = starting_binder
for i in range(iterations):
  y = label_sequence(seq)    
  bo_results.append(y)
  boa.tell(key, seq,label = y['score'])  
  print(f'Tell#{i} {seq} score={y["score"]} rmsd={y["rmsd"]} plddt={y["plddt"]}')
  if i < iterations - 1:
    aq = 'ucb'    
  else:
    aq = 'max'
  if variable_length:
    score = None
    for l in range(len(seq) - 1, len(seq) + 2):
      if l < min_peptide_length:
        continue
      seq_l, score_l = boa.ask(key, aq_fxn=aq, length=l)
      if score is None or score_l > score:
        seq = seq_l
        score = score_l
  else:
    seq, score = boa.ask(key, aq_fxn=aq)  
  print(f'Ask#{i} L={len(seq)}  {seq} {score[0]}')

y = label_sequence(seq)    
print(f'Tell#{i} {seq} score={y["score"]} rmsd={y["rmsd"]} plddt={y["plddt"]}')
bo_results.append(y)
boa.tell(key, seq,label = y['score'])  

In [ ]:
#@title Plot Results

x = np.arange(len(bo_results))
y1 = [r['rmsd'] for r in bo_results]
y2 = [r['plddt'] for r in bo_results]
y3 = [r['score'] for r in bo_results]
ax1 = plt.gca()
ax1.plot(x, y1, label='RMSD')
ax1.set_ylabel('RSMD')
ax2 = plt.twinx()
ax2.plot(x, y2, color='C1', label='pLDDT')
ax2.set_ylabel('pLDDT')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper center')
plt.savefig(f'{basejobname}_bo.png')
plt.show()

plt.plot(x, y3, label='Score')
plt.ylabel('Score')
plt.savefig(f'{basejobname}_bo_score.png')
plt.legend()
plt.show()

with open(f'{basejobname}_bo.json', 'w') as f, open(f'{basejobname}_bo.txt', 'w') as g:
  out = []
  for i, seq in enumerate(boa.seqs):
    # exploiting cache
    r = label_sequence(seq)
    out.append({'step': i, **r})
    if i == 0:
      g.write('step,sequence' + ','.join(r.keys()) +'\n')
    g.write(f'{i},{seq}' + ','.join([str(v) for v in r.values()]) + '\n')
    print(f'#{i:2d}. {seq:20} {r["rmsd"]:2.1f} {r["plddt"]:3.1f} {r["score"]:1.3f}')
  json.dump(out, f, indent=True)

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).



if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

!zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png" $basejobname"_bo"*
files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

## Deatiled AlphaFold Instructions <a name="Instructions"></a>
**Quick start**
1. Paste your protein sequence(s) in the input field.
2. Press "Runtime" -> "Run all".
3. The pipeline consists of 5 steps. The currently running step is indicated by a circle with a stop sign next to it.

**Result zip file contents**

1. PDB formatted structures sorted by avg. pLDDT and complexes are sorted by pTMscore. (unrelaxed and relaxed if `use_amber` is enabled).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. A `predicted_aligned_error_v1.json` using [AlphaFold-DB's format](https://alphafold.ebi.ac.uk/faq#faq-7) and a `scores.json` for each model which contains an array (list of lists) for PAE, a list with the average pLDDT and the pTMscore.
7. BibTeX file with citations for all used tools and databases.

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Additionally, if the `save_to_google_drive` option was selected, the `jobname.result.zip` will be uploaded to your Google Drive.

**MSA generation for complexes**

For the complex prediction we use unpaired and paired MSAs. Unpaired MSA is generated the same way as for the protein structures prediction by searching the UniRef100 and environmental sequences three iterations each.

The paired MSA is generated by searching the UniRef100 database and pairing the best hits sharing the same NCBI taxonomic identifier (=species or sub-species). We only pair sequences if all of the query sequences are present for the respective taxonomic identifier.

**Using a custom MSA as input**

To predict the structure with a custom MSA (A3M formatted): (1) Change the `msa_mode`: to "custom", (2) Wait for an upload box to appear at the end of the "MSA options ..." box. Upload your A3M. The first fasta entry of the A3M must be the query sequence without gaps. 

It is also possilbe to proide custom MSAs for complex predictions. Read more about the format [here](https://github.com/sokrypton/ColabFold/issues/76).

As an alternative for MSA generation the [HHblits Toolkit server](https://toolkit.tuebingen.mpg.de/tools/hhblits) can be used. After submitting your query, click "Query Template MSA" -> "Download Full A3M". Download the A3M file and upload it in this notebook.

**Using custom templates** <a name="custom_templates"></a>

To predict the structure with a custom template (PDB or mmCIF formatted): (1) change the `template_mode` to "custom" in the execute cell and (2) wait for an upload box to appear at the end of the "Input Protein" box. Select and upload your templates (multiple choices are possible).

* Templates must follow the four letter PDB naming.

* Templates in mmCIF format must contain `_entity_poly_seq`. An error is thrown if this field is not present. The field `_pdbx_audit_revision_history.revision_date` is automatically generated if it is not present.

* Templates in PDB format are automatically converted to the mmCIF format. `_entity_poly_seq` and `_pdbx_audit_revision_history.revision_date` are automatically generated.

If you encounter problems, please report them to this [issue](https://github.com/sokrypton/ColabFold/issues/177).

**Comparison to the full AlphaFold2 and Alphafold2 colab**

This notebook replaces the homology detection and MSA pairing of AlphaFold2 with MMseqs2. For a comparison against the [AlphaFold2 Colab](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb) and the full [AlphaFold2](https://github.com/deepmind/alphafold) system read our [preprint](https://www.biorxiv.org/content/10.1101/2021.08.15.456425v1). 

**Troubleshooting**
* Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* Computing resources: Our MMseqs2 API can handle ~20-50k requests per day.
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or MGnify.
* We recommend to additionally use the full [AlphaFold2 pipeline](https://github.com/deepmind/alphafold).

**Description of the plots**
*   **Number of sequences per position** - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.

**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/sokrypton/ColabFold/issues

**License**

The source code of ColabFold is licensed under [MIT](https://raw.githubusercontent.com/sokrypton/ColabFold/main/LICENSE). Additionally, this notebook uses the AlphaFold2 source code and its parameters licensed under [Apache 2.0](https://raw.githubusercontent.com/deepmind/alphafold/main/LICENSE) and [CC BY 4.0](https://creativecommons.org/licenses/by-sa/4.0/) respectively. Read more about the AlphaFold license [here](https://github.com/deepmind/alphafold).

**Acknowledgments**
- We thank the AlphaFold team for developing an excellent model and open sourcing the software. 

- [KOBIC](https://kobic.re.kr) and [Söding Lab](https://www.mpinat.mpg.de/soeding) for providing the computational resources for the MMseqs2 MSA server.

- Richard Evans for helping to benchmark the ColabFold's Alphafold-multimer support.

- [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin, without whom these notebooks would be quite boring!

- Do-Yoon Kim for creating the ColabFold logo.

- A colab by Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton)), Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) and Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)).
